In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

In [2]:
df = pd.read_csv('iot23_train_binary.csv')

In [3]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = pd.get_dummies(df['label']).values

In [4]:
scaler = MinMaxScaler()
scaler.fit(X)
normalized_x = scaler.transform(X)
scaler.fit(Y)
normalized_y = scaler.transform(Y)
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [5]:
class DenseNet(nn.Module):
    def __init__(self, num_classes=2):
        super(DenseNet, self).__init__()

        self.fc1 = nn.Linear(24, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(0.25)

        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.25)

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.25)

        self.fc4 = nn.Linear(256, num_classes)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)

        out = self.fc2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout2(out)

        out = self.fc3(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.dropout3(out)

        out = self.fc4(out)
        out = self.softmax(out)

        return out



# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32)
X_val = torch.tensor(X_test, dtype=torch.float32)
y_val = torch.tensor(Y_test, dtype=torch.float32)

# Initialize the model and optimizer
model = DenseNet(num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    y_pred = model(X_train)
    loss = nn.CrossEntropyLoss()(y_pred, y_train)
    loss.backward()
    optimizer.step()

    # Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        y_pred = model(X_val)
        _, normal_y_val = torch.max(y_val.data,1)
        val_loss = nn.CrossEntropyLoss()(y_pred, y_val)
        val_acc = (y_pred.argmax(1) == normal_y_val).float().mean()

    print(f"Epoch {epoch+1}: Train Loss={loss.item():.4f} Val Loss={val_loss.item():.4f} Val Acc={val_acc.item():.4f}")

Epoch 1: Train Loss=0.6953 Val Loss=0.6926 Val Acc=0.5038
Epoch 2: Train Loss=0.6833 Val Loss=0.6924 Val Acc=0.5038
Epoch 3: Train Loss=0.6740 Val Loss=0.6923 Val Acc=0.5038
Epoch 4: Train Loss=0.6671 Val Loss=0.6921 Val Acc=0.5038
Epoch 5: Train Loss=0.6620 Val Loss=0.6918 Val Acc=0.5038
Epoch 6: Train Loss=0.6590 Val Loss=0.6915 Val Acc=0.5038
Epoch 7: Train Loss=0.6565 Val Loss=0.6912 Val Acc=0.5038
Epoch 8: Train Loss=0.6549 Val Loss=0.6908 Val Acc=0.5490
Epoch 9: Train Loss=0.6534 Val Loss=0.6903 Val Acc=0.5490
Epoch 10: Train Loss=0.6524 Val Loss=0.6898 Val Acc=0.5486
Epoch 11: Train Loss=0.6514 Val Loss=0.6892 Val Acc=0.5485
Epoch 12: Train Loss=0.6506 Val Loss=0.6885 Val Acc=0.5485
Epoch 13: Train Loss=0.6498 Val Loss=0.6878 Val Acc=0.5484
Epoch 14: Train Loss=0.6492 Val Loss=0.6870 Val Acc=0.5484
Epoch 15: Train Loss=0.6483 Val Loss=0.6861 Val Acc=0.5484
Epoch 16: Train Loss=0.6477 Val Loss=0.6852 Val Acc=0.5440
Epoch 17: Train Loss=0.6468 Val Loss=0.6842 Val Acc=0.5439
Epoch 

In [6]:
torch.save(model,'densenet_binary.pt')